In [ ]:
!nvidia-smi

Fri Aug 18 06:48:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
# %cd /content/drive/MyDrive/Colab Notebooks
%cd /content/drive/MyDrive/Colab Notebooks/U-2-Net

/content
/content/drive/MyDrive/Colab Notebooks/U-2-Net


In [ ]:
# !git clone https://github.com/xuebinqin/U-2-Net.git
!pip install --upgrade pip
!pip install -r requirements.txt
!pip install -U pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 MB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for paddlepaddle from https://files.pythonhosted.org/packages/c3/f0/0d418e741d3498d06bb3caa0b428a78c0cd805f968ae012e87e052c61b1e/paddlepaddle-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.3 MB/s eta 0:00:00
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/58/c8/d155c02c382

In [ ]:
# Data augmentation
import os
# print(os.listdir("../input"))
# Input data files are available in the "../input/" directory.
# Any results you write to the current directory are saved as output.

import numpy as np
#import pandas as pd # pd.read_csv
import cv2
import random
import time

class Data_augmentation:
    def __init__(self, path, image_name):
        '''
        Import image
        :param path: Path to the image
        :param image_name: image name
        '''
        self.path = path
        self.name = image_name
        print(os.path.join(path, image_name))
        # self.image = cv2.imread(os.path.join(path, image_name), 0)  ### 1 chanel label image, if 3 channel image, remove "0"
        self.image = cv2.imread(os.path.join(path, image_name))

    def rotate(self, image, angle=90, scale=1.0):
        '''
        Rotate the image
        :param image: image to be processed
        :param angle: Rotation angle in degrees. Positive values mean counter-clockwise rotation (the coordinate origin is assumed to be the top-left corner).
        :param scale: Isotropic scale factor.
        '''
        w = image.shape[1]
        h = image.shape[0]
        #rotate matrix
        M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
        #rotate
        image = cv2.warpAffine(image,M,(w,h))
        return image

    def flip(self, image, vflip=False, hflip=False):
        '''
        Flip the image
        :param image: image to be processed
        :param vflip: whether to flip the image vertically
        :param hflip: whether to flip the image horizontally
        '''
        if hflip or vflip:
            if hflip and vflip:
                c = -1
            else:
                c = 0 if vflip else 1
            image = cv2.flip(image, flipCode=c)
        return image

    def add_GaussianNoise(self, img):
      print("add_GaussianNoise")
      row,col,ch= img.shape
      mean = 0
      var = 0.1
      sigma = var**0.5
      gauss = np.random.normal(mean,sigma,(row,col,ch))
      gauss = gauss.reshape(row,col,ch)
      noisy = img + gauss
      return noisy

    def image_augment(self, save_path):
        '''
        Create the new image with imge augmentation
        :param path: the path to store the new image
        '''
        img = self.image.copy()
        img_flip = self.flip(img, vflip=True, hflip=False)
        img_rot = self.rotate(img)
        img_gaussian = self.add_GaussianNoise(img)  ### 3 channel image
        # img_gaussian = self.image.copy()  ### 1 channel label image

        name_int = self.name[:len(self.name)-4]
        print(name_int)
        cv2.imwrite(save_path+'%s' %str(name_int)+'_vflip.JPG', img_flip)
        cv2.imwrite(save_path+'%s' %str(name_int)+'_rot.JPG', img_rot)
        cv2.imwrite(save_path+'%s' %str(name_int)+'_GaussianNoise.JPG', img_gaussian)

In [ ]:
## data augmetation

def main(file_dir,output_dir):
  file_names = os.listdir(file_dir)

  for i, file_name in enumerate(file_names):
    # file_path = os.path.join(file_dir, file_name)
    # out_path = os.path.join(output_dir, file_name)
    # print(file_path)
    print(i)
    raw_image = Data_augmentation(file_dir, file_name)
    raw_image.image_augment(output_dir)
    time.sleep(0.1)


file_dir = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/train/B"
output_dir = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/train/B_aug/"
main(file_dir=file_dir, output_dir=output_dir)

In [ ]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import os
from PIL import Image
from skimage import io, transform
from scipy import ndimage
import cv2

from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET
from model import U2NETP
# import keras.backend as K

In [ ]:
# ------- 1. define loss function --------

bce_loss = nn.BCELoss(size_average=True)

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = bce_loss(d0,labels_v)
	loss1 = bce_loss(d1,labels_v)
	loss2 = bce_loss(d2,labels_v)
	loss3 = bce_loss(d3,labels_v)
	loss4 = bce_loss(d4,labels_v)
	loss5 = bce_loss(d5,labels_v)
	loss6 = bce_loss(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	# print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

# normalize the predicted SOD probability map
def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

def save_output(image_path,pred,d_dir):

	predict = pred
	predict = predict.squeeze()
	predict_np = predict.cpu().data.numpy()

	im = Image.fromarray(predict_np*255).convert('RGB')
	img_name = image_path.split(os.sep)[-1]
	image = io.imread(image_path)
	imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

	pb_np = np.array(imo)

	out_path = os.path.join(d_dir, img_name)
	print(out_path)
	imo.save(out_path)

# Dice similarity function
def dice_coef(pred, true, k = 1):
	intersection = np.sum(pred[true==k]) * 2.0
	dice = intersection / (np.sum(pred) + np.sum(true))
	return dice

def validate(dataloader, save_result=False, prediction_dir="result/test", img_paths=[]):
	dice_scores = []
		# --------- 4. inference for each image ---------
	for i_test, data_test in enumerate(dataloader):
		inputs_test = data_test['image']
		labels_test = data_test['label']
		inputs_test = inputs_test.type(torch.FloatTensor)
		# labels_test = labels_test.type(torch.FloatTensor)
		if torch.cuda.is_available():
				inputs_test = Variable(inputs_test.cuda())
		else:
				inputs_test = Variable(inputs_test)

		d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

		# normalization
		pred = d1[:,0,:,:]
		pred = normPRED(pred)
		if save_result:
			if not os.path.exists(prediction_dir):
					os.makedirs(prediction_dir, exist_ok=True)
			save_output(img_paths[i_test],pred,prediction_dir)

		pred = (pred.cpu().detach().numpy()[0]*255).astype("uint8")
		pred = pred > 5
		gt = labels_test.cpu().detach().numpy()[0][0]*255
		gt = gt > 5
		dice_score = dice_coef(pred, gt, k = 1)
		# print(dice_score)
		dice_scores.append(dice_score)

	val_dice_score = sum(dice_scores)/len(dice_scores)
	del d1,d2,d3,d4,d5,d6,d7
	print("avg_dice_score: ", val_dice_score)
	return val_dice_score


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# ------- 2. set the directory of training dataset --------

model_name = 'u2net' #'u2netp'

data_dir = os.path.join(os.getcwd(), 'dataset/train' + os.sep)
tra_image_dir = os.path.join(data_dir, "B" + os.sep)
tra_label_dir = os.path.join(data_dir, "B_label" + os.sep)
print(tra_image_dir)
image_ext = '.JPG'
label_ext = '.JPG'

tra_img_name_list = glob.glob(tra_image_dir + '*' + image_ext)
tra_img_name_list.sort()
print("len(tra_img_name_list): ", len(tra_img_name_list))
tra_lbl_name_list = []
for img_path in tra_img_name_list:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(tra_label_dir, img_name)
  tra_lbl_name_list.append(label_path)

### Aug dataset
tra_image_dir_aug = os.path.join(data_dir, "B_aug" + os.sep)
tra_label_dir_aug = os.path.join(data_dir, "B_label_aug" + os.sep)
image_ext = '.JPG'
label_ext = '.JPG'

tra_img_name_list_aug = glob.glob(tra_image_dir_aug + '*' + image_ext)
tra_img_name_list_aug.sort()
print("len(tra_img_name_list_aug): ", len(tra_img_name_list_aug))
tra_lbl_name_list_aug = []
for img_path in tra_img_name_list_aug:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(tra_label_dir_aug, img_name)
  tra_lbl_name_list_aug.append(label_path)

## Val data
val_data_dir = os.path.join(os.getcwd(), 'dataset/validate' + os.sep)
val_image_dir = os.path.join(val_data_dir, "B" + os.sep)
val_label_dir = os.path.join(val_data_dir, "B_label" + os.sep)
print(val_data_dir)
image_ext = '.JPG'
label_ext = '.JPG'

val_img_name_list = glob.glob(val_image_dir + '*' + image_ext)
val_img_name_list.sort()
print("len(val_img_name_list): ", len(val_img_name_list))
val_lbl_name_list = []
for img_path in val_img_name_list:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(val_label_dir, img_name)
  val_lbl_name_list.append(label_path)

## Test data
test_data_dir = os.path.join(os.getcwd(), 'dataset/test' + os.sep)
test_image_dir = os.path.join(test_data_dir, "B" + os.sep)
test_label_dir = os.path.join(test_data_dir, "B_label" + os.sep)
print(test_data_dir)
image_ext = '.JPG'
label_ext = '.JPG'

test_img_name_list = glob.glob(test_image_dir + '*' + image_ext)
test_img_name_list.sort()
print("len(test_img_name_list): ", len(test_img_name_list))
test_lbl_name_list = []
for img_path in test_img_name_list:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(test_label_dir, img_name)
  test_lbl_name_list.append(label_path)


## combine data
tra_img_name_list.extend(tra_img_name_list_aug)
tra_lbl_name_list.extend(tra_lbl_name_list_aug)

print("\n---")
print("train images: ", len(tra_img_name_list))
print("train labels: ", len(tra_lbl_name_list))
print("val images: ", len(val_img_name_list))
print("val labels: ", len(val_lbl_name_list))
# print(val_img_name_list)
# print(val_lbl_name_list)
print("---\n")

save_model_dir = os.path.join(os.getcwd(), 'saved_models/02_01/')
print(save_model_dir)
epoch_num = 100000
batch_size_train = 8
batch_size_val = 1
train_num = 0
val_num = 0

train_num = len(tra_img_name_list)
salobj_dataset = SalObjDataset(
    img_name_list=tra_img_name_list,
    lbl_name_list=tra_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))
salobj_dataloader = DataLoader(salobj_dataset, batch_size=batch_size_train, shuffle=True, num_workers=1)

val_salobj_dataset = SalObjDataset(img_name_list = val_img_name_list,
                                    lbl_name_list = val_lbl_name_list,
                                    transform=transforms.Compose([RescaleT(320),
                                                                  ToTensorLab(flag=0)])
                                    )
val_salobj_dataloader = DataLoader(val_salobj_dataset,
																			batch_size=1,
																			shuffle=False,
																			num_workers=1)

test_salobj_dataset = SalObjDataset(img_name_list = test_img_name_list,
                                    lbl_name_list = test_lbl_name_list,
                                    transform=transforms.Compose([RescaleT(320),
                                                                  ToTensorLab(flag=0)])
                                    )
test_salobj_dataloader = DataLoader(test_salobj_dataset,
																			batch_size=1,
																			shuffle=False,
																			num_workers=1)

/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/train/B/
len(tra_img_name_list):  146
len(tra_img_name_list_aug):  438
/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/
len(val_img_name_list):  32
/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/
len(test_img_name_list):  32

---
train images:  584
train labels:  584
val images:  32
val labels:  32
---

/content/drive/MyDrive/Colab Notebooks/U-2-Net/saved_models/02_01/


In [ ]:
# ------- 3. define model --------
model_dir = "pretrain/u2net.pth"
# model_dir = "saved_models/u2net_bce_itr_700_train_0.515322_tar_0.024292.pth"
# define the net
if(model_name=='u2net'):
    net = U2NET(3, 1)
elif(model_name=='u2netp'):
    net = U2NETP(3,1)

# if torch.cuda.is_available():
#     net.cuda()
if torch.cuda.is_available():
    print("----Loading model....  ", model_dir)
    net.load_state_dict(torch.load(model_dir))
    net.cuda()
else:
    print("----Loading model....  ", model_dir)
    net.load_state_dict(torch.load(model_dir, map_location='cpu'))

# ------- 4. define optimizer --------
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


----Loading model....   pretrain/u2net.pth
---define optimizer...


In [ ]:
# ------- 5. training process --------
print("---start training...")
epoch_num = 1000
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
save_frq = 200 # save the model every 2000 iterations

for epoch in range(0, epoch_num):
  net.train()

  for i, data in enumerate(salobj_dataloader):
    ite_num = ite_num + 1
    ite_num4val = ite_num4val + 1

    inputs, labels = data['image'], data['label']

    inputs = inputs.type(torch.FloatTensor)
    labels = labels.type(torch.FloatTensor)

    # wrap them in Variable
    if torch.cuda.is_available():
        inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                    requires_grad=False)
    else:
        inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

    # y zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
    loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v)

    loss.backward()
    optimizer.step()

    # # print statistics
    running_loss += loss.data.item()
    running_tar_loss += loss2.data.item()

    # del temporary outputs and loss
    del d0, d1, d2, d3, d4, d5, d6, loss2, loss
    if ite_num % 10==0:
        print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
    epoch + 1, epoch_num, (i + 1) * batch_size_train, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

    if ite_num % save_frq == 0:
      print("\n Validating train set...........")
      train_dice_score = validate(salobj_dataloader)
      print("train_dice_score at ite_num is: ", train_dice_score)

      print("\n Validating validate set...........")
      val_dice_score = validate(val_salobj_dataloader)
      print("val_dice_score at ite_num is: ", val_dice_score)

      print("Save model..........")
      print(save_model_dir + "itr_%d_train_%3f_tar_%3f_traindice_%3f_valdice_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val, train_dice_score,val_dice_score))

      torch.save(net.state_dict(), save_model_dir + "itr_%d_train_%3f_tar_%3f_traindice_%3f_valdice_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val, train_dice_score,val_dice_score))
      running_loss = 0.0
      running_tar_loss = 0.0
      net.train()  # resume train
      ite_num4val = 0

---start training...


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch:   1/1000, batch:    80/  584, ite: 10] train loss: 24.584292, tar: 3.613272 
[epoch:   1/1000, batch:   160/  584, ite: 20] train loss: 16.389277, tar: 2.465484 
[epoch:   1/1000, batch:   240/  584, ite: 30] train loss: 12.252635, tar: 1.850029 
[epoch:   1/1000, batch:   320/  584, ite: 40] train loss: 9.722180, tar: 1.448630 
[epoch:   1/1000, batch:   400/  584, ite: 50] train loss: 8.066874, tar: 1.177058 
[epoch:   1/1000, batch:   480/  584, ite: 60] train loss: 6.869368, tar: 0.990622 
[epoch:   1/1000, batch:   560/  584, ite: 70] train loss: 5.998252, tar: 0.855829 
[epoch:   2/1000, batch:    56/  584, ite: 80] train loss: 5.343832, tar: 0.754369 
[epoch:   2/1000, batch:   136/  584, ite: 90] train loss: 4.832706, tar: 0.675099 
[epoch:   2/1000, batch:   216/  584, ite: 100] train loss: 4.415938, tar: 0.611068 
[epoch:   2/1000, batch:   296/  584, ite: 110] train loss: 4.071306, tar: 0.558463 
[epoch:   2/1000, batch:   376/  584, ite: 120] train loss: 3.784626, t

KeyboardInterrupt: ignored

In [ ]:
del net
import gc
torch.cuda.empty_cache()
gc.collect()

29

In [ ]:
###### Inference
## load model
# ------- 3. define model --------
model_name = 'u2net'
model_dir = "saved_models/02_01/itr_1800_train_0.510995_tar_0.024458_traindice_0.965860_valdice_0.949902.pth"
# define the net
if(model_name=='u2net'):
    net = U2NET(3, 1)
elif(model_name=='u2netp'):
    net = U2NETP(3,1)

# if torch.cuda.is_available():
#     net.cuda()
if torch.cuda.is_available():
    print("----Loading model..CUDA..  ", model_dir)
    net.load_state_dict(torch.load(model_dir))
    net.cuda()
else:
    print("----Loading model....  ", model_dir)
    net.load_state_dict(torch.load(model_dir, map_location='cpu'))

# ------- 4. define optimizer --------
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


----Loading model..CUDA..   saved_models/02_01/itr_1800_train_0.510995_tar_0.024458_traindice_0.965860_valdice_0.949902.pth
---define optimizer...


In [ ]:
## Val data
val_data_dir = os.path.join(os.getcwd(), 'dataset/validate' + os.sep)
val_image_dir = os.path.join(val_data_dir, "B" + os.sep)
val_label_dir = os.path.join(val_data_dir, "B_label" + os.sep)
print(val_data_dir)
image_ext = '.JPG'
label_ext = '.JPG'

val_img_name_list = glob.glob(val_image_dir + '*' + image_ext)
val_img_name_list.sort()
print(val_img_name_list)
print("len(val_img_name_list): ", len(val_img_name_list))
val_lbl_name_list = []
for img_path in val_img_name_list:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(val_label_dir, img_name)
  val_lbl_name_list.append(label_path)

val_salobj_dataset = SalObjDataset(img_name_list = val_img_name_list,
                                    lbl_name_list = val_lbl_name_list,
                                    transform=transforms.Compose([RescaleT(320),
                                                                  ToTensorLab(flag=0)])
                                    )
val_salobj_dataloader = DataLoader(val_salobj_dataset,
																			batch_size=1,
																			shuffle=False,
																			num_workers=1)

/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/
['/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204224.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204227.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204233.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204234.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204239.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204240.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204241.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204242.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204254.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204257.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B/204258.1.K.JPG', '/content/

In [ ]:
validate(val_salobj_dataloader, save_result=True, prediction_dir="result/18_08/validate", img_paths=val_img_name_list)

In [ ]:
## Test data
test_data_dir = os.path.join(os.getcwd(), 'dataset/test' + os.sep)
test_image_dir = os.path.join(test_data_dir, "B" + os.sep)
test_label_dir = os.path.join(test_data_dir, "B_label" + os.sep)
print(test_data_dir)
image_ext = '.JPG'
label_ext = '.JPG'

test_img_name_list = glob.glob(test_image_dir + '*' + image_ext)
test_img_name_list.sort()
print(test_img_name_list)
print("len(test_img_name_list): ", len(test_img_name_list))
test_lbl_name_list = []
for img_path in test_img_name_list:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(test_label_dir, img_name)
  test_lbl_name_list.append(label_path)

test_salobj_dataset = SalObjDataset(img_name_list = test_img_name_list,
                                    lbl_name_list = test_lbl_name_list,
                                    transform=transforms.Compose([RescaleT(320),
                                                                  ToTensorLab(flag=0)])
                                    )
test_salobj_dataloader = DataLoader(test_salobj_dataset,
																			batch_size=1,
																			shuffle=False,
																			num_workers=1)

/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/
['/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204290.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204291.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204306.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204309.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204311.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204312.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204323.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204324.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204325.1.K.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204326.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B/204327.1.k.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/te

In [ ]:
validate(test_salobj_dataloader, save_result=True, prediction_dir="result/18_08/test", img_paths=test_img_name_list)

In [ ]:
## Train data
data_dir = os.path.join(os.getcwd(), 'dataset/train' + os.sep)
tra_image_dir = os.path.join(data_dir, "B" + os.sep)
tra_label_dir = os.path.join(data_dir, "B_label" + os.sep)
print(tra_image_dir)
image_ext = '.JPG'
label_ext = '.JPG'

tra_img_name_list = glob.glob(tra_image_dir + '*' + image_ext)
tra_img_name_list.sort()
print("len(tra_img_name_list): ", len(tra_img_name_list))
tra_lbl_name_list = []
for img_path in tra_img_name_list:
  img_name = img_path.split(os.sep)[-1]
  label_path = os.path.join(tra_label_dir, img_name)
  tra_lbl_name_list.append(label_path)

train_salobj_dataset = SalObjDataset(img_name_list = tra_img_name_list,
                                    lbl_name_list = tra_lbl_name_list,
                                    transform=transforms.Compose([RescaleT(320),
                                                                  ToTensorLab(flag=0)])
                                    )
train_salobj_dataloader = DataLoader(train_salobj_dataset,
																			batch_size=1,
																			shuffle=False,
																			num_workers=1)

/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/train/B/
len(tra_img_name_list):  146


In [ ]:
validate(train_salobj_dataloader, save_result=True, prediction_dir="result/18_08/train", img_paths=tra_img_name_list)

In [ ]:
### Seperate nst from image

def separate_white_areas(image_path, save_directory):
  # Open the image
  image = Image.open(image_path).convert('L')  # Convert to grayscale

  # Create a binary image with white areas as 255 and black areas as 0
  binary_image = image.point(lambda x: 255 if x > 128 else 0, '1')

  # Convert the binary image to a NumPy array
  binary_array = np.array(binary_image)

  # Label connected components in the binary image
  labeled_array, num_labels = ndimage.label(binary_array)

  # # Create the save directory if it doesn't exist
  # if not os.path.exists(save_directory):
  #   os.makedirs(save_directory, exist_ok=True)

  separated_images = []
 # Save each separated image in the directory
  for label in range(1, num_labels + 1):
    # Create a mask for the current label
    mask = (labeled_array == label)
    # Create a new image with black background
    separated_image = Image.new('L', image.size, 0)

    # Create a white foreground image with the mask
    foreground = Image.fromarray(mask.astype(np.uint8)*255)

    # Paste the white area onto the black background
    separated_image.paste(foreground, (0, 0), mask=foreground)

    # # Generate the file path for the separated image
    # file_path = os.path.join(save_directory, f'region_{label}.png')

    # Save the separated image
    separated_images.append((np.asarray(separated_image)/255).astype('uint8'))
  return separated_images

def crop_size_standard(mask_full):
  ## Find bounding box
  # print(cv_img.shape)
  h,w = mask_full.shape
  segmentation = np.where(mask_full == 1)
  # Bounding Box
  bbox = 0, 0, 0, 0
  if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
    xmin = int(np.min(segmentation[1]))
    xmax = int(np.max(segmentation[1]))
    ymin = int(np.min(segmentation[0]))
    ymax = int(np.max(segmentation[0]))

    bbox = xmin, xmax, ymin, ymax
    ori_x_min, ori_x_max, ori_y_min, ori_y_max = xmin, xmax, ymin, ymax
    print("bbox: ", bbox)
    chromo_w = xmax - xmin
    chromo_h = ymax - ymin
    if chromo_w < 256:
        padding = (256- (xmax-xmin))/2
        if padding == int(padding):
            padding_left = padding_right = int(padding)
        else:
            padding_left = int(padding)
            padding_right = padding_left +1

        xmin = xmin - padding_left
        if xmin < 0:
            padding_right = padding_right -xmin
            xmin = 0
        xmax = xmax + padding_right
        if xmax > w:
            xmin = xmin - (xmax-w)
            xmax = w

    if chromo_h < 256:
        padding = (256- (ymax-ymin))/2
        if padding == int(padding):
            padding_top = padding_btm = int(padding)
        else:
            padding_top = int(padding)
            padding_btm = padding_top +1

        ymin = ymin - padding_top
        if ymin < 0:
            padding_btm = padding_btm - ymin
            ymin = 0
        ymax = ymax + padding_btm
        if ymax > h:
            ymin = ymin - (ymax - h)
            ymax = h

  print(xmax-xmin, ymax-ymin)
  print(xmax, xmin, ymax, ymin)
  mask_full = mask_full[ymin:ymax, xmin: xmax]
  return mask_full, xmin, xmax, ymin, ymax

In [ ]:
image_directory = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/validate/B"
label_directory = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result/18_08/validate"

# Specify the directory where you want to save the separated images
save_directory = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate"
if not os.path.exists(save_directory):
  os.makedirs(save_directory, exist_ok=True)
img_names = os.listdir(image_directory)
img_names.sort()

for img_name in img_names:
  if img_name.endswith(".JPG") or img_name.endswith(".jpg"):
    # Đường dẫn đầy đủ tới tệp tin ảnh
    image_path = os.path.join(image_directory, img_name)
    label_path = os.path.join(label_directory, img_name)
    print(image_path)

    img = cv2.imread(image_path)
    # Tạo thư mục riêng cho từng tệp tin ảnh
    image_save_directory = os.path.join(save_directory, os.path.splitext(img_name)[0])
    if not os.path.exists(image_save_directory):
      os.makedirs(image_save_directory, exist_ok=True)

    # Tách ảnh và lưu các ảnh đã tách riêng vào thư mục riêng
    separated_images = separate_white_areas(label_path, image_save_directory)
    if separated_images == []:
      print(f"NO nst found: {label_path}")
    else:
      for index, separated_image in enumerate(separated_images):
        print(f"\tCropping Nst {index}")
        mask_full_single_cluster_chrom, xmin, xmax, ymin, ymax = crop_size_standard(separated_image)
        chromos_img = img.copy()
        chromos_img = chromos_img[ymin:ymax, xmin: xmax]
        chromos_img[mask_full_single_cluster_chrom == 0] = (255,255,255)
        cv2.imwrite(os.path.join(image_save_directory, img_name.split('.JP')[0] + "_" + str(index) + "_" + "img" + ".JPG"), chromos_img)
        cv2.imwrite(os.path.join(image_save_directory, img_name.split('.JP')[0] + "_" + str(index) + "_" + "full" + ".JPG"), mask_full_single_cluster_chrom*255)
    # break

In [ ]:
image_directory = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/dataset/test/B"
label_directory = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result/18_08/test"

# Specify the directory where you want to save the separated images
save_directory = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test"
if not os.path.exists(save_directory):
  os.makedirs(save_directory, exist_ok=True)
img_names = os.listdir(image_directory)
img_names.sort()

for img_name in img_names:
  if img_name.endswith(".JPG") or img_name.endswith(".jpg"):
    # Đường dẫn đầy đủ tới tệp tin ảnh
    image_path = os.path.join(image_directory, img_name)
    label_path = os.path.join(label_directory, img_name)
    print(image_path)

    img = cv2.imread(image_path)
    # Tạo thư mục riêng cho từng tệp tin ảnh
    image_save_directory = os.path.join(save_directory, os.path.splitext(img_name)[0])
    if not os.path.exists(image_save_directory):
      os.makedirs(image_save_directory, exist_ok=True)

    # Tách ảnh và lưu các ảnh đã tách riêng vào thư mục riêng
    separated_images = separate_white_areas(label_path, image_save_directory)
    if separated_images == []:
      print(f"NO nst found: {label_path}")
    else:
      for index, separated_image in enumerate(separated_images):
        print(f"\tCropping Nst {index}")
        mask_full_single_cluster_chrom, xmin, xmax, ymin, ymax = crop_size_standard(separated_image)
        chromos_img = img.copy()
        chromos_img = chromos_img[ymin:ymax, xmin: xmax]
        chromos_img[mask_full_single_cluster_chrom == 0] = (255,255,255)
        cv2.imwrite(os.path.join(image_save_directory, img_name.split('.JP')[0] + "_" + str(index) + "_" + "img" + ".JPG"), chromos_img)
        cv2.imwrite(os.path.join(image_save_directory, img_name.split('.JP')[0] + "_" + str(index) + "_" + "full" + ".JPG"), mask_full_single_cluster_chrom*255)
    # break

In [ ]:
### ChromSeg region_guided_UNetplus
%cd /content/drive/MyDrive/Colab Notebooks/chromseg/region_guided_UNetplus
!pwd

/content/drive/MyDrive/Colab Notebooks/chromseg/region_guided_UNetplus
/content/drive/MyDrive/Colab Notebooks/chromseg/region_guided_UNetplus


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import Dataset
import torchvision.models as models
import PIL.Image as Image
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import copy
from loss import *
from UNet_plus import UNet_plus2 as UNet_plus2
from utils import *
import argparse
import sys
from utils import x_transforms, y_transforms

In [ ]:
seed = 1
model_type = "UNet_plus2"
model_path = "models/0530_UNet_plus2_original_dataset_seed1_IoU-nan_IoU-0.7401_gamma-1_alpha-0.75.pth"
IMAGE_SIZE = 256
batch_size = 8

torch.manual_seed(seed)    # reproducible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if model_type == 'UNet_plus2':
    print('UNet_plus2')
    model = UNet_plus2(3, 1)
    # model = model.to(device)
else:
    raise Exception('Invalid model type: %s'%model_type)

# import pdb; pdb.set_trace()
ckpt = torch.load(model_path)
model.load_state_dict(ckpt)
model.eval()
model = model.to(device)

def pre_process(x_path):
    img_x = Image.open(x_path)
    if img_x.size != (IMAGE_SIZE,IMAGE_SIZE):
        img_x = img_x.resize((IMAGE_SIZE, IMAGE_SIZE))
    img_x = x_transforms(img_x)
    img_x = img_x.unsqueeze(0)
    return img_x


cuda
UNet_plus2


In [ ]:
dataset = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate"
save_output = "result/18_08/validate"

folder_names = os.listdir(dataset)
folder_names.sort()

for i, folder_name in enumerate(folder_names):
  print(f"\n{i}. {folder_name}")
  folder_path = os.path.join(dataset, folder_name)
  img_names = os.listdir(folder_path)
  img_names = [name for name in img_names if 'img' in name.split('_')[-1]]
  img_paths = [os.path.join(folder_path, name) for name in img_names]
  print(img_paths[:5])

  output_ol_folder = os.path.join(save_output, folder_name, "overlap")
  output_nonol_folder = os.path.join(save_output, folder_name, "non_overlap")

  if not os.path.exists(output_ol_folder):
    os.makedirs(output_ol_folder, exist_ok=True)
  if not os.path.exists(output_nonol_folder):
    os.makedirs(output_nonol_folder, exist_ok=True)

  with torch.no_grad():
    for id, img_path in enumerate(img_paths):
      img_name = img_names[id]
      x = pre_process(img_path).to(device)
      y = model(x)
      y_pred_0 = torch.squeeze(y[0].cpu()).numpy()
      y_pred = np.zeros((256,256))
      y_pred[y_pred_0 > 0.5] = 1.0
      y_2_0 = torch.squeeze(y[1].cpu()).numpy()
      y_2 = np.zeros((256,256))
      y_2[y_2_0 > 0.5] = 1.0
      output1 = np.reshape(y_pred * 255,(256,256))
      output2 = np.reshape(y_2 * 255,(256,256))

      x_image = torch.squeeze(x.cpu()).numpy()
      image = np.dstack((x_image[0,...]*255, x_image[1,...]*255, x_image[2,...]*255))

      cv2.imwrite(os.path.join(output_ol_folder, img_name), output1)
      cv2.imwrite(os.path.join(output_nonol_folder, img_name), output2)
  # if i > 1:
  #   break
      # cv2.imwrite('mask/' + str(n) + ".png", image)


0. 204224.1.k
['/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204224.1.k/204224.1.k_0_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204224.1.k/204224.1.k_1_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204224.1.k/204224.1.k_2_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204224.1.k/204224.1.k_3_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204224.1.k/204224.1.k_4_img.JPG']

1. 204227.1.k
['/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204227.1.k/204227.1.k_0_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204227.1.k/204227.1.k_1_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate/204227.1.k/204227.1.k_2_img.JPG', '/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/

In [ ]:
dataset = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test"
save_output = "result/18_08/test"

folder_names = os.listdir(dataset)
folder_names.sort()

for i, folder_name in enumerate(folder_names):
  print(f"\n{i}. {folder_name}")
  folder_path = os.path.join(dataset, folder_name)
  img_names = os.listdir(folder_path)
  img_names = [name for name in img_names if 'img' in name.split('_')[-1]]
  img_paths = [os.path.join(folder_path, name) for name in img_names]
  print(img_paths[:5])

  output_ol_folder = os.path.join(save_output, folder_name, "overlap")
  output_nonol_folder = os.path.join(save_output, folder_name, "non_overlap")

  if not os.path.exists(output_ol_folder):
    os.makedirs(output_ol_folder, exist_ok=True)
  if not os.path.exists(output_nonol_folder):
    os.makedirs(output_nonol_folder, exist_ok=True)

  with torch.no_grad():
    for id, img_path in enumerate(img_paths):
      img_name = img_names[id]
      x = pre_process(img_path).to(device)
      y = model(x)
      y_pred_0 = torch.squeeze(y[0].cpu()).numpy()
      y_pred = np.zeros((256,256))
      y_pred[y_pred_0 > 0.5] = 1.0
      y_2_0 = torch.squeeze(y[1].cpu()).numpy()
      y_2 = np.zeros((256,256))
      y_2[y_2_0 > 0.5] = 1.0
      output1 = np.reshape(y_pred * 255,(256,256))
      output2 = np.reshape(y_2 * 255,(256,256))

      x_image = torch.squeeze(x.cpu()).numpy()
      image = np.dstack((x_image[0,...]*255, x_image[1,...]*255, x_image[2,...]*255))

      cv2.imwrite(os.path.join(output_ol_folder, img_name), output1)
      cv2.imwrite(os.path.join(output_nonol_folder, img_name), output2)
  # if i > 1:
  #   break
      # cv2.imwrite('mask/' + str(n) + ".png", image)

In [ ]:
### ChromSeg crossing_partition
%cd /content/drive/MyDrive/Colab Notebooks/chromseg/crossing_partition
!pwd

/content/drive/MyDrive/Colab Notebooks/chromseg/crossing_partition
/content/drive/MyDrive/Colab Notebooks/chromseg/crossing_partition


In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import copy
from skimage import io
import math
import copy
import time
from utils2 import *

# from utils2 import crossing_reconstruct
import traceback

In [ ]:
dataset = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/validate"
base_region_guided_UNetplus_path = "/content/drive/MyDrive/Colab Notebooks/chromseg/region_guided_UNetplus/result/18_08/validate"
BASE_OUTPUT_PATH = "result_partition/18_08/validate"
IMAGE_SIZE = 256
IMG_SIZE = 256

if not os.path.isdir(BASE_OUTPUT_PATH):
    os.makedirs(BASE_OUTPUT_PATH)

folder_names = os.listdir(dataset)
folder_names.sort()

for i, folder_name in enumerate(folder_names):
  print(f"\n{i}. {folder_name}")
  folder_path = os.path.join(dataset, folder_name)
  img_names = os.listdir(folder_path)
  img_names = [name for name in img_names if 'img' in name.split('_')[-1]]
  img_paths = [os.path.join(folder_path, name) for name in img_names]
  # print(img_paths[:5])

  OVERLAP_PATH = os.path.join(base_region_guided_UNetplus_path, folder_name, "overlap")
  NON_OVERLAP_PATH = os.path.join(base_region_guided_UNetplus_path, folder_name, "non_overlap")
  for id, img_path in enumerate(img_paths):
      print("\n", img_path)
      img_name = img_names[id]
      # print(os.path.join(OVERLAP_PATH, img_name))
      image = cv2.imread(img_path)
      overlapped = cv2.imread(os.path.join(OVERLAP_PATH, img_name), 0)
      non_overlapped = cv2.imread(os.path.join(NON_OVERLAP_PATH, img_name), 0)
      # print(type(overlapped), type(non_overlapped))
      # print(overlapped.shape, non_overlapped.shape)
      if image.shape != (IMAGE_SIZE,IMAGE_SIZE,3):
          # print(0)
          image = cv2.resize(image,(IMAGE_SIZE, IMAGE_SIZE))
      if overlapped.shape != (IMAGE_SIZE,IMAGE_SIZE):
          # print(1)
          overlapped = cv2.resize(overlapped, (IMAGE_SIZE, IMAGE_SIZE))
      if non_overlapped.shape != (IMAGE_SIZE,IMAGE_SIZE):
          # print(2)
          non_overlapped = cv2.resize(non_overlapped,(IMAGE_SIZE, IMAGE_SIZE))

      # overlapped[overlapped == 255] = 1
      # non_overlapped[non_overlapped == 255] = 1

      overlapped[overlapped <= 100] = 0
      overlapped[overlapped > 100] = 1
      non_overlapped[non_overlapped <= 100] = 0
      non_overlapped[non_overlapped > 100] = 1

      output = crossing_reconstruct(image, overlapped, non_overlapped)
      OUTPUT_PATH = os.path.join(BASE_OUTPUT_PATH, folder_name)
      output_path = OUTPUT_PATH
      if not os.path.exists(output_path):
          os.makedirs(output_path)

      # cv2.imwrite(os.path.join(output_path, "crossing_" + img_name), image)
      overlapped_vis = cv2.cvtColor(overlapped*255, cv2.COLOR_GRAY2BGR)
      non_overlapped_vis = cv2.cvtColor(non_overlapped*255, cv2.COLOR_GRAY2BGR)
      results = [image, overlapped_vis, non_overlapped_vis]
      try:
          gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
          for idx, mask in enumerate(output):
              # print(idx)
              img = copy.deepcopy(gray)
              new_mask = np.zeros((IMG_SIZE, IMG_SIZE))
              for i in range(0, IMG_SIZE):
                  for j in range(0, IMG_SIZE):
                      if(mask[i,j] == 1):
                          for x in range(-1,2):
                              for y in range(-1,2):
                                  try:
                                      new_mask[i+x,j+y] = 1
                                  except:
                                      continue

              img[(new_mask == 0)] = 255
              results.append(cv2.cvtColor(img,cv2.COLOR_GRAY2BGR))
              # cv2.imwrite(os.path.join(output_path, str(num)+".png"), img)
          h_concat = cv2.hconcat(results)
          cv2.imwrite(os.path.join(output_path, img_name), h_concat)
      except:
          raise "error(fail to partition)"
  # if i>0:
  #   break

In [ ]:
dataset = "/content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test"
base_region_guided_UNetplus_path = "/content/drive/MyDrive/Colab Notebooks/chromseg/region_guided_UNetplus/result/18_08/test"
BASE_OUTPUT_PATH = "result_partition/18_08/test"
IMAGE_SIZE = 256
IMG_SIZE = 256

if not os.path.isdir(BASE_OUTPUT_PATH):
    os.makedirs(BASE_OUTPUT_PATH)

folder_names = os.listdir(dataset)
folder_names.sort()

for i, folder_name in enumerate(folder_names):
  print(f"\n{i}. {folder_name}")
  folder_path = os.path.join(dataset, folder_name)
  img_names = os.listdir(folder_path)
  img_names = [name for name in img_names if 'img' in name.split('_')[-1]]
  img_paths = [os.path.join(folder_path, name) for name in img_names]
  # print(img_paths[:5])

  OVERLAP_PATH = os.path.join(base_region_guided_UNetplus_path, folder_name, "overlap")
  NON_OVERLAP_PATH = os.path.join(base_region_guided_UNetplus_path, folder_name, "non_overlap")
  for id, img_path in enumerate(img_paths):
      print("\n", img_path)
      img_name = img_names[id]
      # print(os.path.join(OVERLAP_PATH, img_name))
      image = cv2.imread(img_path)
      overlapped = cv2.imread(os.path.join(OVERLAP_PATH, img_name), 0)
      non_overlapped = cv2.imread(os.path.join(NON_OVERLAP_PATH, img_name), 0)
      # print(type(overlapped), type(non_overlapped))
      # print(overlapped.shape, non_overlapped.shape)
      if image.shape != (IMAGE_SIZE,IMAGE_SIZE,3):
          # print(0)
          image = cv2.resize(image,(IMAGE_SIZE, IMAGE_SIZE))
      if overlapped.shape != (IMAGE_SIZE,IMAGE_SIZE):
          # print(1)
          overlapped = cv2.resize(overlapped, (IMAGE_SIZE, IMAGE_SIZE))
      if non_overlapped.shape != (IMAGE_SIZE,IMAGE_SIZE):
          # print(2)
          non_overlapped = cv2.resize(non_overlapped,(IMAGE_SIZE, IMAGE_SIZE))

      # overlapped[overlapped == 255] = 1
      # non_overlapped[non_overlapped == 255] = 1

      overlapped[overlapped <= 100] = 0
      overlapped[overlapped > 100] = 1
      non_overlapped[non_overlapped <= 100] = 0
      non_overlapped[non_overlapped > 100] = 1

      output = crossing_reconstruct(image, overlapped, non_overlapped)
      OUTPUT_PATH = os.path.join(BASE_OUTPUT_PATH, folder_name)
      output_path = OUTPUT_PATH
      if not os.path.exists(output_path):
          os.makedirs(output_path)

      # cv2.imwrite(os.path.join(output_path, "crossing_" + img_name), image)
      overlapped_vis = cv2.cvtColor(overlapped*255, cv2.COLOR_GRAY2BGR)
      non_overlapped_vis = cv2.cvtColor(non_overlapped*255, cv2.COLOR_GRAY2BGR)
      results = [image, overlapped_vis, non_overlapped_vis]
      try:
          gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
          for idx, mask in enumerate(output):
              # print(idx)
              img = copy.deepcopy(gray)
              new_mask = np.zeros((IMG_SIZE, IMG_SIZE))
              for i in range(0, IMG_SIZE):
                  for j in range(0, IMG_SIZE):
                      if(mask[i,j] == 1):
                          for x in range(-1,2):
                              for y in range(-1,2):
                                  try:
                                      new_mask[i+x,j+y] = 1
                                  except:
                                      continue

              img[(new_mask == 0)] = 255
              results.append(cv2.cvtColor(img,cv2.COLOR_GRAY2BGR))
              # cv2.imwrite(os.path.join(output_path, str(num)+".png"), img)
          h_concat = cv2.hconcat(results)
          cv2.imwrite(os.path.join(output_path, img_name), h_concat)
      except:
          raise "error(fail to partition)"
  # if i>0:
  #   break


0. 204290.1.K

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_0_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_1_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_2_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_3_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_4_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_5_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_6_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_7_img.JPG

 /content/drive/MyDrive/Colab Notebooks/U-2-Net/result_separate/18_08/test/204290.1.K/204290.1.K_8_img.JPG

 /content/dr

In [ ]:
type(overlapped)

NoneType

In [ ]:
img_path = os.path.join(OVERLAP_PATH, img_name)
print(img_path)
print(os.path.exists(img_path))
overlapped = cv2.imread(img_path, 0)

/content/drive/MyDrive/Colab Notebooks/chromseg/region_guided_UNetplus/result/18_08/test/204290.1.K/overlap/204290.1.K_0_img.JPG
True


In [ ]:
type(overlapped)

numpy.ndarray

In [ ]:
image_dir = os.path.join(os.getcwd(), 'dataset/train', 'B')
prediction_dir = os.path.join(os.getcwd(), 'dataset/train', model_name + '_results' + os.sep)
img_name_list = glob.glob(image_dir + os.sep + '*')
print(img_name_list)
print("len(img_name_list): ", len(img_name_list))

# --------- 2. dataloader ---------
#1. dataloader
test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                    lbl_name_list = [],
                                    transform=transforms.Compose([RescaleT(320),
                                                                  ToTensorLab(flag=0)])
                                    )
test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                    batch_size=1,
                                    shuffle=False,
                                    num_workers=1)

# --------- 4. inference for each image ---------
for i_test, data_test in enumerate(test_salobj_dataloader):

    print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

    inputs_test = data_test['image']
    inputs_test = inputs_test.type(torch.FloatTensor)

    if torch.cuda.is_available():
        inputs_test = Variable(inputs_test.cuda())
    else:
        inputs_test = Variable(inputs_test)

    d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

    # normalization
    pred = d1[:,0,:,:]
    pred = normPRED(pred)

    # save results to test_results folder
    if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir, exist_ok=True)
    save_output(img_name_list[i_test],pred,prediction_dir)

    del d1,d2,d3,d4,d5,d6,d7